LSTM Нейросеть - Игорь

In [1]:
# Для нейросети
import sys
sys.path.append('..')
from config import Config
from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Flatten, Activation, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
%matplotlib inline

In [2]:
import os
os.listdir('..')
    
xTrain = np.load(Config.DATASET_X_TRAIN)
yTrain = np.load(Config.DATASET_Y_TRAIN)
xTest = np.load(Config.DATASET_X_TEST)
yTest = np.load(Config.DATASET_Y_TEST)
# xVal, yVal = xTrain[:10000], yTrain[:10000]
# xTrain, yTrain = xTrain[10000:], yTrain[10000:]
# xBuffer, yBuffer = xTest, yTest
# xTest, yTest = xVal, yVal
# xVal, yVal = xBuffer, yBuffer
# # для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
# print("Размерности тренировочного набора")
# print(xTrain.shape)
# print(yTrain.shape)
# print()
# print("Размерности тестового набора")
# print(xTest.shape)
# print(yTest.shape)
# print()
# # print("Размерности val набора")
# # print(xVal.shape)
# # print(yVal.shape)

In [6]:
numWords = 20000
xLen = 50

In [7]:
modelLSTM = Sequential()

modelLSTM.add(Embedding(numWords, 100, input_length=xLen))
modelLSTM.add(SpatialDropout1D(0.2))
modelLSTM.add(BatchNormalization())

modelLSTM.add(LSTM(10))
modelLSTM.add(Dense(2, activation='softmax'))

modelLSTM.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

modelLSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           2000000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 50, 100)          0         
 lDropout1D)                                                     
                                                                 
 batch_normalization_1 (Batc  (None, 50, 100)          400       
 hNormalization)                                                 
                                                                 
 lstm_1 (LSTM)               (None, 10)                4440      
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                                 
Total params: 2,004,862
Trainable params: 2,004,662
No

In [8]:
# Обучаем сеть
Batch_Size = 256

historyLSTM = modelLSTM.fit(xTrain, 
                            yTrain, 
                            epochs=15,
                            batch_size=Batch_Size,
                            # validation_data=(xVal, yVal),
                            use_multiprocessing=True)

Epoch 1/15
264/264 [==============================] - 10s 34ms/step - loss: 0.4877 - accuracy: 0.7926
Epoch 2/15
264/264 [==============================] - 10s 38ms/step - loss: 0.2514 - accuracy: 0.8973
Epoch 3/15
264/264 [==============================] - 10s 39ms/step - loss: 0.1279 - accuracy: 0.9568
Epoch 4/15
264/264 [==============================] - 10s 39ms/step - loss: 0.0759 - accuracy: 0.9774
Epoch 5/15
264/264 [==============================] - 10s 38ms/step - loss: 0.0526 - accuracy: 0.9850
Epoch 6/15
264/264 [==============================] - 10s 40ms/step - loss: 0.0351 - accuracy: 0.9904
Epoch 7/15
264/264 [==============================] - 10s 39ms/step - loss: 0.0284 - accuracy: 0.9926
Epoch 8/15
264/264 [==============================] - 11s 42ms/step - loss: 0.0186 - accuracy: 0.9953
Epoch 9/15
264/264 [==============================] - 11s 42ms/step - loss: 0.0174 - accuracy: 0.9952
Epoch 10/15
264/264 [==============================] - 10s 39ms/step - loss: 0.012

In [10]:

# Проверка точности на тестовой выборке
rightAnswer = [0,0]
totalAnswer = [0,0]

# Получаем результаты распознавания для каждого класса
currPred = modelLSTM.predict(xTest)
# Определяем номер распознанного класса для каждого вектора
currOut = np.argmax(currPred, axis=1)
# Определяем правильный класс для каждого вектора
yOut = np.argmax(yTest, axis=1)

print(currPred.shape)
print(currOut.shape)
print(yOut.shape)
print()

for i in range(len(yOut)):
  predictA = currOut[i]
  rightA   = yOut[i]

  totalAnswer[rightA] += 1
  if predictA == rightA:
    rightAnswer[rightA] += 1

print("Точность распознавания текстов")
for i in range(2):
  print("{:12s}: {:3d} из {:3d} - {:3.2f}%".format(str(i), rightAnswer[i], totalAnswer[i], (rightAnswer[i]/totalAnswer[i]*100)))

(868, 2)
(868,)
(868,)

Точность распознавания текстов
0           : 608 из 701 - 86.73%
1           :  22 из 167 - 13.17%
